In [1]:
import pandas as pd
import re

In [2]:
landsat5 = pd.read_csv('./LANDSAT_ETM_C1_282311.csv', encoding = "ISO-8859-1", parse_dates=['Acquisition Date'])
landsat5["Acquisition Date"] = pd.to_datetime(landsat5['Acquisition Date'].dt.strftime("%Y%m%d"))

In [3]:
landsat5.shape

(442, 71)

In [4]:
landsat5['Acquisition Date'].dt.year.unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999])

In [5]:
landsat5.sort_values(by='Acquisition Date', inplace=True)
landsat5.reset_index(inplace=True)
landsat5.rename(mapper={"Acquisition Date": "date"}, axis=1, inplace=True)

In [6]:
landsat5_gs = pd.DataFrame(columns=['area', 'date', 'b', 'gs_location'])

In [12]:
land5_lst = []
with open('landsat7.txt', 'r') as file:
    for line in file:
        rex = re.compile(r'.TIF')
        if rex.search(line) != None:
            land5_lst.append(line)
land5_lst[::-5]

['gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180929_20180929_01_RT/gap_mask/LE07_L1TP_012031_20180929_20180929_01_RT_GM_B8.TIF.gz\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180929_20180929_01_RT/gap_mask/LE07_L1TP_012031_20180929_20180929_01_RT_GM_B4.TIF.gz\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180929_20180929_01_RT/LE07_L1TP_012031_20180929_20180929_01_RT_B8.TIF\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180929_20180929_01_RT/LE07_L1TP_012031_20180929_20180929_01_RT_B4.TIF\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180828_20180923_01_T1/gap_mask/LE07_L1TP_012031_20180828_20180923_01_T1_GM_B7.TIF.gz\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180828_20180923_01_T1/gap_mask/LE07_L1TP_012031_20180828_20180923_01_T1_GM_B3.TIF.gz\n',
 'gs://gcp-public-data-landsat/LE07/01/012/031/LE07_L1TP_012031_20180828_20180923_01_T1/LE07_L1TP_01

In [13]:
for idx, line in enumerate(land5_lst):
    dic = {}
    spl = line.split("_")
    dic["area"] = spl[2][:3]
    dic["date"] = spl[3]
    dic["b"] = spl[-1].split(".")[0]
    dic["gs_location"] = line
    landsat5_gs.loc[idx] = dic

In [14]:
landsat5_gs.shape

(8101, 4)

In [15]:
landsat5_gs = landsat5_gs[(landsat5_gs.b == 'B4') | (landsat5_gs.b == 'B3')]
landsat5_gs.reset_index(inplace=True)
landsat5_gs["date"] = pd.to_datetime(landsat5_gs["date"])

In [16]:
landsat5_merged = landsat5_gs.merge(landsat5, on="date")

In [17]:
landsat5_merged.isnull().sum().sum()

2182

In [20]:
landsat5_merged[landsat5_merged.isnull()]

,index_x,area,date,b,gs_location,index_y,Landsat Product Identifier,Landsat Scene Identifier,Spacecraft Identifier,Scan Line Corrector,...,UL Corner Long dec,UR Corner Lat dec,UR Corner Long dec,LL Corner Lat dec,LL Corner Long dec,LR Corner Lat dec,LR Corner Long dec,Display ID,Ordering ID,Browse Link
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
landsat5_merged.gs_location = landsat5_merged.gs_location.str.replace("\n", "")

In [19]:
with open('landsat7_link.txt', 'w') as f:
    for item in landsat5_merged.gs_location:
        f.write("%s\n" % item)